In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(work_dir+'/fgc_support_retri/')
from fgc_support_retri.train import *
from fgc_support_retri.utils import *
from fgc_support_retri import config

I0327 00:34:47.596624 140318771083072 file_utils.py:39] PyTorch version 1.1.0 available.


In [3]:
train_entity_match_model(10, 12, "20200326_ematch_old_etype_lr=2e-5", 2e-5)

I0327 00:35:01.110204 140318771083072 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0327 00:35:02.280414 140318771083072 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0327 00:35:02.283610 140318771083072 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

dev_set indexing...


  0%|          | 0/882 [00:00<?, ?it/s]

train_set indexing...
{'text': '電視名人。為川普集團前任董事長兼總裁', 'text_cn': '电视名人。为川普集团前任董事长兼总裁', 'start': 76, 'end': 94}
{'text': '川普娛樂公司的創辦人，在全世界經營房地產', 'text_cn': '川普娱乐公司的创办人，在全世界经营房地产', 'start': 95, 'end': 115}
{'text': '熱，電，輻射', 'text_cn': '热，电，辐射', 'start': 22, 'end': 28}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}
{'text': '校園注意教室之空氣流通，落實勤洗手，有呼吸道症狀時應配戴口罩；打噴嚏時應用面紙', 'text_cn': '校园注意教室之空气流通，落实勤洗手，有呼吸道症狀时应配戴口罩；打喷嚏时应用面纸', 'start': 88, 'end': 127}
{'text': '他人交談時，儘可能保持1公尺以上之距離，避免病毒傳播。', 'text_cn': '他人交谈时，尽可能保持1公尺以上之距离，避免病毒传播。', 'start': 143, 'end': 170}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}


  0%|          | 0/2619 [00:00<?, ?it/s]

...

{'sp_em': 0.142, 'sp_prec': 0.608, 'sp_recall': 0.56, 'sp_f1': 0.529}
epoch 8 eval_recall: 0.560 eval_f1: 0.529


  1%|          | 2/239 [00:00<00:12, 19.15it/s]

epoch 9 train_loss: 0.004


100%|██████████| 239/239 [00:20<00:00, 11.94it/s]


{'sp_em': 0.134, 'sp_prec': 0.612, 'sp_recall': 0.566, 'sp_f1': 0.533}
epoch 9 eval_recall: 0.566 eval_f1: 0.533


# Revise shint

In [10]:
data = json_load(config.FGC_TEST)
get_SHINT(data)
get_answer_sp(data)

100%|██████████| 193/193 [00:00<00:00, 21879.58it/s]

{'text': '不具入侵能力但失控繁殖的细胞，称为良性肿瘤', 'start': 492, 'end': 513}
{'text': '牠们会在夜晚成群的爬出洞穴，循著海浪的声音，利用海面光线的导引，快速的爬向大海的家', 'start': 431, 'end': 472}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，', 'start': 58, 'end': 104}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，却不能像海鸟一样飞翔在蔚蓝的天空。', 'start': 58, 'end': 121}


In [11]:
config.FGC_TEST

PosixPath('/work/fgc_support_retri/data/FGC/FGC_release_1.7.8/FGC_release_all_test.json')

In [12]:
json_write(data, 'FGC_release_all_test.json')